In [1]:
#import all PySpark related libraries

from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator


import pandas as pd
from pandas.tools.plotting import scatter_matrix

In [3]:
#create an instance of a Spark session
#add an app_name

spark = SparkSession.builder.appName('bin-class-spark').getOrCreate()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2767562570638957> in <module> () 
 2 #add an app_name 
 3 
 ----> 4 spark = SparkSession . builder . appName ( 'bin-class-spark' ) . getOrCreate ( ) 

 NameError : name 'SparkSession' is not defined

In [4]:
#creating dataframe from CSV file
#Add the csv file to the DBFS and then you cna refer it from there.

df = spark.read.csv('/FileStore/tables/bank.csv', header=True, inferSchema=True)   

In [5]:
#printing the schema of created dataframe 
df.printSchema()  

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- deposit: string (nullable = true)

In [6]:
#extracting numeric features from dataframe, where i is used to iterate 'int' type members of dataframe 
numeric_features = [i[0] for i in df.dtypes if i[1] == 'int']      

In [7]:
#print numeric_features 
numeric_features

Out[ 61 ]: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [8]:
#convert numeric features of database into a Pandas dataframe, which is a very memory intensive process as it is run on only one executer  
numeric_data = df.select(numeric_features).toPandas()

#create a visual of a scatter matrix from the numeric_data 
axs = pd.scatter_matrix(numeric_data, figsize=(8,8))

#count number of coloumns of the numeric_data dataframe  
num_cols = len(numeric_data.columns)

#iterating through all columns in axs & plotting it
for i in range(num_cols):
  v = axs[i,0]
  v.yaxis.label.set_rotation(0)
  v.yaxis.label.set_ha('right')
  v.set_yticks(())
  h=axs[num_cols-1, i]
  h.xaxis.label.set_rotation(90)
  h.set_xticks(())

In [9]:
#array of features 
features = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit']

#create a dataframe of columns 
cols = df.columns

#create a dataframe using the features array
df = df[features]

In [10]:
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']

stages = []

#iterating through categorical columns to modify the dataframe 
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [11]:
#creating a pipeline of all stages & fitting it on the dataframe 
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)

#printing relevant schema of the transformed dataframe 
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
-- label: double (nullable = false)
-- features: vector (nullable = true)
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- deposit: string (nullable = true)

In [12]:
#display top five rows of dataframe 
df.show(5)

+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
label| features|age| job|marital|education|default|balance|housing|loan|contact|duration|campaign|pdays|previous|poutcome|deposit|
+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
 1.0|(30,[3,11,13,16,1...| 59| admin.|married|secondary| no| 2343| yes| no|unknown| 1042| 1| -1| 0| unknown| yes|
 1.0|(30,[3,11,13,16,1...| 56| admin.|married|secondary| no| 45| no| no|unknown| 1467| 1| -1| 0| unknown| yes|
 1.0|(30,[2,11,13,16,1...| 41|technician|married|secondary| no| 1270| yes| no|unknown| 1389| 1| -1| 0| unknown| yes|
 1.0|(30,[4,11,13,16,1...| 55| services|married|secondary| no| 2476| yes| no|unknown| 579| 1| -1| 0| unknown| yes|
 1.0|(30,[3,11,14,16,1...| 54| admin.|married| tertiary| no| 184| no| no|unknown| 673| 2| -1| 0| unknown| yes|
+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
only showing top 5 rows

In [13]:
#split the dataset into train and test data (70:30) with seed for stability 
train, test = df.randomSplit([0.7, 0.3], seed = 2019)

#print number of traning and testing records 
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7797
Test Dataset Count: 3365

In [14]:
#import GBTClassifier model 
from pyspark.ml.classification import GBTClassifier

#create instance of GBTClassifier 
gbt = GBTClassifier(maxIter=10)

#train the GBTClassifier model on the train data 
gbtModel = gbt.fit(train)

#test the trained model on test data 
predictions = gbtModel.transform(test)

#show projected predicitions made by trained model 
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
age| job|label| rawPrediction|prediction| probability|
+---+----------+-----+--------------------+----------+--------------------+
 35|management| 0.0|[0.00523863003964...| 0.0|[0.50261929105925...|
 37|management| 0.0|[0.45611759846541...| 0.0|[0.71345733533148...|
 52|management| 0.0|[-0.6139332433376...| 1.0|[0.22655505859473...|
 53|management| 0.0|[0.03698770815621...| 0.0|[0.51848542493511...|
 57|management| 0.0|[0.45611759846541...| 0.0|[0.71345733533148...|
 61|management| 0.0|[-0.3904664207042...| 1.0|[0.31411887221274...|
 46|management| 0.0|[1.31338130936222...| 0.0|[0.93256424129022...|
 36|management| 0.0|[0.98079391560200...| 0.0|[0.87670468947647...|
 40|management| 0.0|[0.76957704719894...| 0.0|[0.82334172179109...|
 40|management| 0.0|[0.97202467792988...| 0.0|[0.87479633368517...|
+---+----------+-----+--------------------+----------+--------------------+
only showing top 10 rows

In [15]:
#testing accuracy of trained model using BinaryClassifierEvaluator 
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.9037395969515111

In [16]:
#cross validate the model by using parameter grid builder 
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

#evaluating our model. 
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)